# Superposition approach

Compute the admittance matrix and solve for an arbitrary current.
This method leads to a singular matrix if no ground is defined.

In [ ]:
import netgen.occ as occ
from netgen.webgui import Draw as DrawGeo
import ngsolve
from ngsolve.webgui import Draw
import numpy as np

In [ ]:
unit_square = occ.unit_square_shape

In [ ]:
e1 = occ.Circle((0.2, 0.2), r=0.1)
e2 = occ.Circle((0.5, 0.5), r=0.1)
e3 = occ.Circle((0.8, 0.8), r=0.1)

In [ ]:
electrodes = [e1.Face(), e2.Face(), e3.Face()]
for idx, electrode in enumerate(electrodes):
    for edge in electrode.edges:
        edge.name = "Electrode_{}".format(idx)
    unit_square = unit_square - electrode

In [ ]:
DrawGeo(unit_square)

In [ ]:
mesh = ngsolve.Mesh(occ.OCCGeometry(unit_square, dim=2).GenerateMesh())
mesh.Curve(2)

In [ ]:
Draw(mesh)
print(mesh.GetBoundaries())

In [ ]:
# 3 - number of electrodes
admittance_matrix = np.zeros((3, 3))

I_1 = -0.5  # A
I_2 = -0.5
I_3 = 1.0
I = [I_1, I_2, I_3]
boundaries = ["Electrode_{}".format(idx) for idx in range(3)]
indices = [(0, 0), (1, 1), (2,2 ), (0, 1), (0, 2), (1, 2)]

def bvp(dirichlet_values, Vi, Vj, Yii, Yjj):
    fes = ngsolve.H1(mesh, order=2, dirichlet=str("|").join(boundaries)) 
    u = fes.TrialFunction()
    v = fes.TestFunction()

    a = ngsolve.BilinearForm(fes)
    a += ngsolve.grad(u) * ngsolve.grad(v) * ngsolve.dx

    f = ngsolve.LinearForm(fes)
    f += 0.0 * v * ngsolve.dx

    a.Assemble()
    f.Assemble()

    gfu = ngsolve.GridFunction(fes)
    bnd_cf = mesh.BoundaryCF(dirichlet_values)
    gfu.Set(bnd_cf, ngsolve.BND)
    Draw(gfu)
    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gfu.vec
    gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    
    power = 0.5 * ngsolve.Integrate(ngsolve.grad(gfu) * ngsolve.grad(gfu) * ngsolve.dx, mesh)
    if np.isclose(Vj, 0):
        return 2.0 / Vi * power
    else:
        return 1.0 / (Vi * Vj)  * power - 0.5 * (Vi / Vj * Yii + Vj / Vi * Yjj)

for index_pair in indices:
    idx_1 = index_pair[0]
    idx_2 = index_pair[1]
    dirichlet_values = {}
    for boundary in boundaries:
        dirichlet_values[boundary] = 0.0
    if idx_1 == idx_2:
        dirichlet_values["Electrode_{}".format(idx_1)] = 1.0
        print(idx_1, idx_2, dirichlet_values)
        admittance_matrix[idx_1][idx_1] = bvp(dirichlet_values, 1.0, 0.0,
                                              admittance_matrix[idx_1][idx_1],
                                              admittance_matrix[idx_2][idx_2])
    else:
        dirichlet_values["Electrode_{}".format(idx_1)] = 1.0
        dirichlet_values["Electrode_{}".format(idx_2)] = 2.0
        print(idx_1, idx_2, dirichlet_values)

        admittance_matrix[idx_1][idx_2] = bvp(dirichlet_values, 1.0, 2.0,
                                              admittance_matrix[idx_1][idx_1],
                                              admittance_matrix[idx_2][idx_2])
        admittance_matrix[idx_2][idx_1] = admittance_matrix[idx_1][idx_2]

In [ ]:
admittance_matrix

# Floating potentials

## Adjust admittance matrix for fixed potential

Fix V_2 (third electrode) to zero.

In [ ]:
V_2 = 0
idx_remove = 2

I_1 = -0.5  # A
I_2 = -0.5
I_3 = 1.0
I = np.array([I_1, I_2, I_3])

I_reduced = np.zeros(I.shape[0] - 1)
for i in range(idx_remove - 1):
    I_reduced[i] = I[i]
I_reduced[idx_remove - 1] = I[idx_remove - 1] - I[idx_remove]
admittance_matrix_reduced = admittance_matrix[:idx_remove,:idx_remove]
admittance_matrix_reduced[idx_remove - 1] += -admittance_matrix[idx_remove, :idx_remove]
V = np.linalg.solve(admittance_matrix_reduced, I_reduced)
print(V)

# Check output

In [ ]:
for idx, boundary in enumerate(boundaries):
    if idx == 2:
        dirichlet_values[boundary] = V_2
    else:
        dirichlet_values[boundary] = V[idx]

fes = ngsolve.H1(mesh, order=2, dirichlet=str("|").join(boundaries)) 
u = fes.TrialFunction()
v = fes.TestFunction()

a = ngsolve.BilinearForm(fes)
a += ngsolve.grad(u) * ngsolve.grad(v) * ngsolve.dx

f = ngsolve.LinearForm(fes)
f += 0.0 * v * ngsolve.dx

a.Assemble()
f.Assemble()

gfu = ngsolve.GridFunction(fes)
bnd_cf = mesh.BoundaryCF(dirichlet_values)
gfu.Set(bnd_cf, ngsolve.BND)
Draw(gfu)
r = f.vec.CreateVector()
r.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
Draw(gfu)

# Field plot

In [ ]:
Draw(-ngsolve.grad(gfu), mesh)

# Result from floating code

Shows that the solution is equal (within a certain numerical accuracy)

In [ ]:
V_float = np.array([-0.727856, -0.515505])
print("Diff: ", V_float - V)